<a href="https://colab.research.google.com/github/anipriyo/ApnaHood-v2/blob/main/QKD_BB84_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [75]:
!pip install qiskit
!pip install qiskit_aer
!pip install pylatexenc

In [76]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer import Aer, AerSimulator

In [77]:

n=100;


def encode(bits,basis):
  publickey=[]
  for i in range(n):
    qc=QuantumCircuit(1,1)
    if basis[i]==0: # z basis encoding 0=0 and 1=1
      if bits[i]==0:
        pass
      else:
        qc.x(0)
    else: # x basis encoding 0= |+> and 1= |->
      if bits[i]==0:
        qc.h(0)
      else:
        qc.x(0)
        qc.h(0)
    qc.barrier()
    publickey.append(qc)
  return publickey

def decode(msg,basis):
  measurements=[]
  for q in range(n):
    if basis[q]==0:
      msg[q].measure(0,0)
    if basis[q]==1:
      msg[q].h(0)
      msg[q].measure(0,0)
    counts=run_circuit(msg[q],shots=1)
    measured_bit=next(iter(counts))
    measurements.append(int(measured_bit))
  return measurements

def remove_garbage(a_basis, b_basis, bits):
    good_bits = []
    for q in range(n):
        if a_basis[q] == b_basis[q]:
            # if a==b then keep bits else reject
            good_bits.append(bits[q])
    return good_bits

def error_induced(bit_list):
    flipped_list = []
    for bit in bit_list:
        flip_prob = random.uniform(0.25, 0.5)  # Random flip rate between 25% and 50%
        if random.random() < flip_prob:
            flipped_bit = 1 - bit  # Flip 0 to 1 or 1 to 0
        else:
            flipped_bit = bit
        flipped_list.append(flipped_bit)
    return flipped_list


def run_circuit(circuit, shots=1):
    """Run a circuit on a simulator and return the counts."""
    simulator = AerSimulator()

    from qiskit import transpile
    transpiled_circuit = transpile(circuit, simulator)
    result = simulator.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()

    return counts

In [78]:
sharedkey=np.random.randint(2,size=n) # <2 that means 0 and 1 only
print(sharedkey)

[1 0 0 1 0 1 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 1
 0 1 0 0 1 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0 1 0 1 1 1 1 1 0 0 0 0 1 1
 1 0 0 1 0 0 0 1 0 1 0 1 0 0 1 1 1 0 1 0 1 0 0 0 1 0]


In [79]:
sender_basis = np.random.randint(2, size=n)
print(sender_basis) # 0 for z basis and 1 for x basis

[1 0 0 0 0 1 0 0 1 0 1 1 0 0 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 1 1 1 1 0 0 0
 0 0 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 1
 1 1 0 1 1 0 1 1 0 1 1 0 0 0 0 1 1 1 0 1 0 1 0 1 1 0]


In [80]:
publickey=encode(sharedkey,sender_basis)
# display(publickey[0])
publickey[0].draw()

┌───┐┌───┐ ░ 
  q: ┤ X ├┤ H ├─░─
     └───┘└───┘ ░ 
c: 1/═════════════

Reciever

In [81]:

recieved_publickey=error_induced(publickey) # interception by eve creates error


reciever_basis_guess = np.random.randint(2, size=n)

decodedkey=decode(publickey,reciever_basis_guess)
print(decodedkey)

NameError: name 'random' is not defined

In [ ]:

reciever_key = remove_garbage(sender_basis, reciever_basis_guess, decodedkey)
print(reciever_key)
sender_key = remove_garbage(sender_basis, reciever_basis_guess, sharedkey)
int_list = [int(x) for x in sender_key]
print(int_list)


In [ ]:
print("Keys match:", reciever_key == int_list)


match_percentage = len(reciever_key) / n * 100
print(f"Percentage of matching bases: {match_percentage:.2f}%")

# eavesdropping detection
def detect_eavesdropping(alice_bits, bob_bits, sample_size=None):
    if sample_size is None:
        sample_size = min(20, len(bob_bits)//4)
    check_positions = np.random.choice(range(len(bob_bits)), sample_size, replace=False)

    alice_sample = [alice_bits[i] for i in check_positions]
    bob_sample = [bob_bits[i] for i in check_positions]


    errors = sum(a != b for a, b in zip(alice_sample, bob_sample))
    error_rate = errors / sample_size


    remaining_positions = [i for i in range(len(bob_bits)) if i not in check_positions]
    alice_final = [alice_bits[i] for i in remaining_positions]
    bob_final = [bob_bits[i] for i in remaining_positions]

    return error_rate, alice_final, bob_final

error_rate, alice_final, bob_final = detect_eavesdropping(int_list, reciever_key)
print(f"Detected error rate: {error_rate:.2f}")
print(f"Final key length: {len(alice_final)}")